- 1st 실습

PCA를 이용해서 diabates 적용해보기
(n수는 자유)

- 2nd AE 실습

(수정된 신용카드 거래 소스올리면, 그걸 해보고 수행한후 응용해서 아래 실습)

1. DSA데이터에서 
lyingRigh                  480
lyingBack                  480
을 정상 (normal) jumping(abnormal)을 비정상 데이터로 사용

2. 두개 (lying+jumping) activity를 섞어서 test 

3. lying만 가지고 훈련하여 AE 구축

4. 테스트 데이터(lying+jumping)AE에 넣어서 Reconstruction error(입력/출력차이)를 구함

5. 적당한 threshold값으로 분류수행

6. accuracy를 구함



In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
# import seaborn as sns
import tensorflow as tf
from tensorflow import keras

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-11-20 23:48:51.502534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from sklearn.decomposition import PCA
df = pd.read_csv("datas/diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)
pca_result

array([[-1.75694707, -1.11174258],
       [ 1.50742148,  0.55940565],
       [-0.65082201, -1.92957633],
       ...,
       [ 0.57408846, -0.03381634],
       [ 0.17269908, -1.32289215],
       [ 1.32126715,  1.02748861]])

In [7]:

pca_df = pd.DataFrame(pca_result)
pca_df.head()

,0,1
0,-1.756947,-1.111743
1,1.507421,0.559406
2,-0.650822,-1.929576
3,1.587398,1.065075
4,-2.483374,2.359563


In [8]:
pca.explained_variance_ratio_

array([0.26138907, 0.19714578])

In [25]:
df = pd.read_csv("datas/DSA_features.csv")
df

,T_xacc_mean,T_xacc_max,T_xacc_min,T_xacc_var,T_xacc_std,T_xacc_skew,T_yacc_mean,T_yacc_max,T_yacc_min,T_yacc_var,...,LL_ymag_std,LL_ymag_skew,LL_zmag_mean,LL_zmag_max,LL_zmag_min,LL_zmag_var,LL_zmag_std,LL_zmag_skew,activity,people
0,7.975714,8.1605,7.6823,0.014395,0.119981,-0.023319,1.083150,1.1832,0.99744,0.002208,...,0.000792,0.177075,-0.057119,-0.054963,-0.059241,6.778722e-07,0.000823,0.036729,sitting,p1
1,7.978250,8.1763,7.8472,0.007551,0.086896,0.552416,1.140865,1.2129,1.05810,0.000784,...,0.000860,-0.286918,-0.057268,-0.054945,-0.059589,7.032302e-07,0.000839,0.347471,sitting,p1
2,7.970894,8.0860,7.8470,0.003092,0.055603,0.100538,1.140962,1.2128,1.07960,0.000508,...,0.000762,-0.134430,-0.057068,-0.054711,-0.059065,6.268222e-07,0.000792,0.045579,sitting,p1
3,7.938412,8.1083,7.6901,0.003763,0.061343,-0.231914,1.165260,1.3170,1.07870,0.002173,...,0.000735,0.021485,-0.056422,-0.053670,-0.058310,8.011245e-07,0.000895,0.240690,sitting,p1
4,7.908930,8.1305,7.8322,0.001741,0.041731,2.042285,1.187504,1.2574,1.09450,0.000662,...,0.000824,-0.148229,-0.055801,-0.053313,-0.057815,6.853423e-07,0.000828,0.258429,sitting,p1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9115,8.280854,34.1980,-2.9038,28.080803,5.299132,1.350075,-1.491537,11.2240,-11.65100,14.670334,...,0.200829,-0.040701,0.297666,0.708480,-0.117430,4.135451e-02,0.203358,-0.310022,basketBall,p8
9116,9.591118,51.6970,-3.4129,35.722025,5.976791,2.981144,0.086304,6.9951,-11.76400,5.329897,...,0.148745,-0.266377,0.224716,0.554670,-0.250950,3.355704e-02,0.183186,-0.736410,basketBall,p8
9117,9.599113,27.9300,-1.0765,48.850886,6.989341,0.449237,-0.728367,3.7801,-8.36910,5.683022,...,0.310748,-0.009505,-0.237786,0.088854,-0.477260,2.026107e-02,0.142341,0.668438,basketBall,p8
9118,9.692482,72.7820,-2.6734,59.378336,7.705734,4.491114,-0.582724,6.1216,-8.85710,4.162963,...,0.156493,0.050624,0.533023,0.677800,0.055941,1.356379e-02,0.116464,-1.482489,basketBall,p8


In [26]:
df.describe()

,T_xacc_mean,T_xacc_max,T_xacc_min,T_xacc_var,T_xacc_std,T_xacc_skew,T_yacc_mean,T_yacc_max,T_yacc_min,T_yacc_var,...,LL_ymag_min,LL_ymag_var,LL_ymag_std,LL_ymag_skew,LL_zmag_mean,LL_zmag_max,LL_zmag_min,LL_zmag_var,LL_zmag_std,LL_zmag_skew
count,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,...,9120.000000,9.120000e+03,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9.120000e+03,9120.000000,9120.000000
mean,7.765766,14.625536,3.602974,17.807013,2.454290,0.305587,-0.811036,1.801847,-3.704204,2.049747,...,0.111347,1.818862e-02,0.105897,-0.055440,0.064504,0.169687,-0.052866,7.624694e-03,0.059342,-0.089861
std,3.758022,11.823448,5.622855,45.188787,3.432895,0.768360,2.201692,4.177165,3.982579,3.908675,...,0.362373,2.382393e-02,0.083518,0.526959,0.346660,0.360190,0.359712,1.903445e-02,0.064059,0.619636
min,-11.575660,-4.758700,-99.715000,0.000135,0.011633,-7.826631,-9.782885,-9.703600,-49.941000,0.000141,...,-0.901760,3.044156e-07,0.000552,-3.693948,-0.912397,-0.910830,-1.126000,2.960874e-07,0.000544,-5.111532
25%,8.374161,9.523200,0.000000,0.036855,0.191977,-0.161535,-1.134192,0.172375,-4.837875,0.012121,...,-0.111835,5.683172e-04,0.023839,-0.343123,-0.156271,-0.035821,-0.326125,2.227543e-04,0.014925,-0.402781
50%,9.186328,12.549500,5.757900,2.241468,1.497153,0.275042,-0.375678,1.571750,-2.470150,0.737663,...,0.184470,1.078612e-02,0.103856,-0.075786,0.086285,0.210825,-0.040447,1.813539e-03,0.042586,-0.092036
75%,9.515573,15.843250,7.649075,6.680571,2.584680,0.845410,0.127612,3.005450,-1.279725,1.900426,...,0.315962,2.740661e-02,0.165549,0.177076,0.295415,0.422270,0.206933,6.927870e-03,0.083234,0.228493
max,10.713139,93.694000,9.749000,392.898630,19.821671,9.070164,7.051530,41.013000,2.656400,54.332000,...,0.969020,3.041940e-01,0.551538,5.035185,1.020691,1.042600,0.999440,2.790198e-01,0.528223,3.560828


In [27]:
df.columns

Index(['T_xacc_mean', 'T_xacc_max', 'T_xacc_min', 'T_xacc_var', 'T_xacc_std',
       'T_xacc_skew', 'T_yacc_mean', 'T_yacc_max', 'T_yacc_min', 'T_yacc_var',
       ...
       'LL_ymag_std', 'LL_ymag_skew', 'LL_zmag_mean', 'LL_zmag_max',
       'LL_zmag_min', 'LL_zmag_var', 'LL_zmag_std', 'LL_zmag_skew', 'activity',
       'people'],
      dtype='object', length=272)

In [28]:
df.isnull().sum()

T_xacc_mean     0
T_xacc_max      0
T_xacc_min      0
T_xacc_var      0
T_xacc_std      0
               ..
LL_zmag_var     0
LL_zmag_std     0
LL_zmag_skew    0
activity        0
people          0
Length: 272, dtype: int64

In [33]:
normal = df[(df['activity'] == 'lyingRigh') | (df['activity'] == 'lyingBack')]
abnormal = df[df['activity'] == 'jumping']

In [34]:
test_data = pd.concat([normal, abnormal])

In [ ]:
lying_data = normal[(normal['activity'] == 'lyingRigh') | (normal['activity'] == 'lyingBack')]

print(lying_data.dtypes)
print(lying_data.head())

T_xacc_mean     float64
T_xacc_max      float64
T_xacc_min      float64
T_xacc_var      float64
T_xacc_std      float64
                 ...   
LL_zmag_var     float64
LL_zmag_std     float64
LL_zmag_skew    float64
activity         object
people           object
Length: 272, dtype: object
     T_xacc_mean  T_xacc_max  T_xacc_min  T_xacc_var  T_xacc_std  T_xacc_skew  \
960    -4.834646     -4.7185     -4.9049    0.001124    0.033529     0.632141   
961    -4.780810     -4.5850     -4.8833    0.003948    0.062835     1.052880   
962    -4.804400     -4.7037     -4.8755    0.001482    0.038496     0.735226   
963    -4.750563     -4.5696     -4.8977    0.004447    0.066683     0.710263   
964    -4.774916     -4.6741     -4.8457    0.001080    0.032868     0.324779   

     T_yacc_mean  T_yacc_max  T_yacc_min  T_yacc_var  ...  LL_ymag_std  \
960    -0.501018    -0.41744    -0.59561    0.001016  ...     0.000986   
961    -0.480601    -0.40372    -0.63184    0.001297  ...     0.000788   


In [ ]:
X_lying = lying_data.drop(columns=['activity'])  
X_lying = lying_data.select_dtypes(include=[np.number])

scaler = StandardScaler()
X_lying_scaled = scaler.fit_transform(X_lying)


In [39]:
from tensorflow.keras import layers, models
input_dim = X_lying_scaled.shape[1]
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(64, activation='relu')(input_layer)
decoded = layers.Dense(input_dim, activation='sigmoid')(encoded)

2024-11-21 00:19:37.136571: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [41]:
autoencoder = models.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(X_lying_scaled, X_lying_scaled, epochs=50, batch_size=256, validation_split=0.2)

Epoch 1/50
3/3 [==============================] - 1s 93ms/step - loss: 1.3085 - val_loss: 1.0124
Epoch 2/50
3/3 [==============================] - 0s 25ms/step - loss: 1.2811 - val_loss: 0.9972
Epoch 3/50
3/3 [==============================] - 0s 25ms/step - loss: 1.2550 - val_loss: 0.9828
Epoch 4/50
3/3 [==============================] - 0s 20ms/step - loss: 1.2282 - val_loss: 0.9681
Epoch 5/50
3/3 [==============================] - 0s 25ms/step - loss: 1.1994 - val_loss: 0.9526
Epoch 6/50
3/3 [==============================] - 0s 20ms/step - loss: 1.1679 - val_loss: 0.9360
Epoch 7/50
3/3 [==============================] - 0s 20ms/step - loss: 1.1351 - val_loss: 0.9181
Epoch 8/50
3/3 [==============================] - 0s 20ms/step - loss: 1.1017 - val_loss: 0.8991
Epoch 9/50
3/3 [==============================] - 0s 20ms/step - loss: 1.0693 - val_loss: 0.8794
Epoch 10/50
3/3 [==============================] - 0s 65ms/step - loss: 1.0392 - val_loss: 0.8593
Epoch 11/50
3/3 [============

In [42]:
X_abnormal = abnormal.drop(columns=['activity'])
X_abnormal = X_abnormal.select_dtypes(include=[np.number])
X_abnormal_scaled = scaler.transform(X_abnormal)

In [43]:
X_lying_reconstructed = autoencoder.predict(X_lying_scaled)
X_abnormal_reconstructed = autoencoder.predict(X_abnormal_scaled)

15/15 [==============================] - 0s 989us/step


In [44]:
lying_reconstruction_error = np.mean(np.square(X_lying_scaled - X_lying_reconstructed), axis=1)
abnormal_reconstruction_error = np.mean(np.square(X_abnormal_scaled - X_abnormal_reconstructed), axis=1)
print(lying_reconstruction_error, abnormal_reconstruction_error)

[  0.19901539   0.22789352   0.1832106    0.23802488   0.21954757
   0.19944803   0.18334948   0.16846507   0.17887697   0.2283504
   0.1835668    0.17893518   0.21652982   0.159471     0.20457288
   0.1779677    0.15718076   0.17471547   0.18815303   0.18263198
   0.2185423    0.15715243   0.209622     0.4640969    0.18419509
   0.17662752   0.18842828   0.40098588   0.63862343   0.57201331
   0.51537121   0.18121433   0.32939599   0.30616305   0.27473608
   0.1585754    0.18356717   0.26274837   0.22641305   0.16584027
   0.16669435   0.18824043   0.3659204    0.19559952   0.30473384
   0.22106128   0.51892649   0.32270955   0.4466923    0.25462974
   0.23119675   0.15939165   0.19640017   0.19400166   0.17003141
   0.17512031   0.179915     0.15311621   0.15556162   0.15382809
   2.36728748 106.85911494   2.94045314   3.7191842    1.58402498
   0.95123137   0.63311794   0.60640942   0.60863818   0.58146759
   1.4963662    0.70547196   0.6757768    0.62523306   0.58839722
   1.527658

In [45]:
threshold = np.percentile(lying_reconstruction_error, 95)

In [ ]:
from sklearn.metrics import accuracy_score
y_true = np.concatenate([np.zeros(len(lying_reconstruction_error)), np.ones(len(abnormal_reconstruction_error))])
y_pred = np.concatenate([
    (lying_reconstruction_error > threshold).astype(int),
    (abnormal_reconstruction_error > threshold).astype(int)
])

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 96.67%
